# To create an LP model for shipment optimization:

Goal: Minimize the total transportation cost.

Objective Function: 𝑍=∑(cost per unit×shipment quantity)

Constraints:

Capacity constraint: Shipment quantity cannot exceed warehouse capacity.

Demand constraint: Shipment should meet the customer demand.

Non-negativity: The shipment quantity cannot be negative.


In [ ]:
# Import necessary libraries
import pandas as pd
from scipy.optimize import linprog

# Load the Excel file
file_path = "Supply chain logisitcs problem.xlsx"
xls = pd.ExcelFile(file_path)

# Load the relevant sheets into DataFrames
order_list = xls.parse('OrderList')
freight_rates = xls.parse('FreightRates')
wh_capacities = xls.parse('WhCapacities')
wh_costs = xls.parse('WhCosts')

# ---- DATA PREPARATION ----

# Merge OrderList with FreightRates to get the cost details
merged_df = order_list.merge(freight_rates, 
                             left_on=['Carrier', 'Origin Port', 'Destination Port'],
                             right_on=['Carrier', 'orig_port_cd', 'dest_port_cd'],
                             how='left')

# Merge with Warehouse Costs
merged_df = merged_df.merge(wh_costs, left_on='Plant Code', right_on='WH', how='left')

# Merge with Warehouse Capacities
merged_df = merged_df.merge(wh_capacities, left_on='Plant Code', right_on='Plant ID', how='left')

# Filter necessary columns
data = merged_df[['Order ID', 'Unit quantity', 'Cost/unit', 'Daily Capacity']].dropna()

# ---- LP MODEL SETUP ----

# Define the objective function coefficients (cost per shipment)
c = data['Cost/unit'].values

# Constraints
A = []  # Coefficient matrix
b = []  # Capacity constraints

# Warehouse capacity constraint
for _, row in data.iterrows():
    A.append([1 if i == _ else 0 for i in range(len(data))])
    b.append(row['Daily Capacity'])

# Bounds for shipment quantity (0 <= x <= demand)
bounds = [(0, row['Unit quantity']) for _, row in data.iterrows()]

# Solve the LP problem
result = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')

# Display the solution
print("Optimal Shipment Quantities:")
for i, quantity in enumerate(result.x):
    print(f"Order {data['Order ID'].iloc[i]}: {quantity:.2f} units")

print("\nMinimum Cost:", result.fun)


In [2]:
# Import necessary libraries
import pandas as pd
from scipy.optimize import linprog

In [3]:
# Load the Excel file
file_path = "C:/Users/user/Downloads/Supply chain logisitcs problem.xlsx"
xls = pd.ExcelFile(file_path)

In [4]:
# Load the relevant sheets into DataFrames
order_list = xls.parse('OrderList')
freight_rates = xls.parse('FreightRates')
wh_capacities = xls.parse('WhCapacities')
wh_costs = xls.parse('WhCosts')

In [12]:
# Merge OrderList with FreightRates to get the cost details
merged_df = order_list.merge(freight_rates, 
                             left_on=['Carrier', 'Origin Port', 'Destination Port'],
                             right_on=['Carrier', 'orig_port_cd', 'dest_port_cd'],
                             how='left')